In [2]:
import requests
from bs4 import BeautifulSoup

headers = {
    'Referer': 'https://health-diet.ru/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}
url = "https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie"
response = requests.get(url, headers=headers)
src = response.text

with open('index.html', "w", encoding="utf-8") as file:
  file.write(src)


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
  
with open('index.html', encoding="utf-8") as file:
  src = file.read()

soup = BeautifulSoup(src, "lxml")
all_products_hrefs = soup.find_all(class_="uk-flex mzr-tc-group-item")

all_categories_dict = {}

for item in all_products_hrefs:
  item_text = item.text.strip()
  item_href = "https://health-diet.ru" + item.find(class_="mzr-tc-group-item-href").get("href")
  
  # Сохраняем в виде словаря и формате json.
  all_categories_dict[item_text] = item_href

# Создаем фаил json.
# indent=4, -Отступ в начале строки в фаиле json.
# ensure_ascii=False -Не экранирует сивал и помагает с кирилицей.
with open("all_categories_dict.json", "w", encoding='utf-8') as file:
    json.dump(all_categories_dict, file, indent=4, ensure_ascii=False)
all_categories_dict

In [ ]:
import random
from time import sleep
import requests
from bs4 import BeautifulSoup
import json
import csv

# url = "https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie"
#
headers = {
    'Referer': 'https://health-diet.ru/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# Загружаем фаил json.
with open("all_categories_dict.json") as file:
    all_categories = json.load(file)

# Информация в консоль
iteration_count = int(len(all_categories)) - 1
count = 0
print(f"Всего итераций: {iteration_count}")


for category_name, category_href in all_categories.items():
    # Список из символов которые мы хотим заменить.
    rep = [",", " ", "-", "'"]
    for item in rep:
        # Если символ в имени есть меняем егт на _
        if item in category_name:
            category_name = category_name.replace(item, "_")
    # Создаем запрос.
    req = requests.get(url=category_href, headers=headers)
    src = req.text

    # Сохраняем страницу в- category_name
    # Добавляем счетчик к имени фаила- count
    with open(f"data/{count}_{category_name}.html", "w", encoding="utf-8") as file:
        file.write(src)
        
    # Открываем фаил для читения.
    with open(f"data/{count}_{category_name}.html", encoding="utf-8") as file:
        src = file.read()

    soup = BeautifulSoup(src, "lxml")

    # проверка страницы на наличие таблицы с продуктами
    alert_block = soup.find(class_="uk-alert-danger")
    if alert_block is not None:
        continue

    # собираем заголовки таблицы
    table_head = soup.find(class_="mzr-tc-group-table").find("tr").find_all("th")
    product = table_head[0].text
    calories = table_head[1].text
    proteins = table_head[2].text
    fats = table_head[3].text
    carbohydrates = table_head[4].text
    # Записываем CSV file
    with open(f"data/{count}_{category_name}.csv", "w", encoding="utf-8") as file:
        writer = csv.writer(file)
        # чтобы записать несколько создаем кортеж
        writer.writerow(
            (
                product,
                calories,
                proteins,
                fats,
                carbohydrates
            )
        )

    # собираем данные продуктов
    products_data = soup.find(class_="mzr-tc-group-table").find("tbody").find_all("tr")

    product_info = []
    for item in products_data:
        product_tds = item.find_all("td")

        title = product_tds[0].find("a").text
        calories = product_tds[1].text
        proteins = product_tds[2].text
        fats = product_tds[3].text
        carbohydrates = product_tds[4].text
        
        # Информация в json file.
        product_info.append(
            {
                "Title": title,
                "Calories": calories,
                "Proteins": proteins,
                "Fats": fats,
                "Carbohydrates": carbohydrates
            }
        )
        
        # Обновляем CSV file
        with open(f"data/{count}_{category_name}.csv", "a", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(
                (
                    title,
                    calories,
                    proteins,
                    fats,
                    carbohydrates
                )
            )

    # сохрняем json file
    with open(f"data/{count}_{category_name}.json", "a", encoding="utf-8") as file:
        json.dump(product_info, file, indent=4, ensure_ascii=False)

    count += 1
    
    # Информация в консоль
    print(f"# Итерация {count}. {category_name} записан...")
    iteration_count = iteration_count - 1

    if iteration_count == 0:
        print("Работа завершена")
        break

    print(f"Осталось итераций: {iteration_count}")
    sleep(random.randrange(2, 4))

Всего итераций: 53
# Итерация 1. Р‘Р°СЂР°РЅРёРЅР°_Рё_РґРёС‡СЊ записан...
Осталось итераций: 52
# Итерация 2. Р‘РѕР±РѕРІС‹Рµ записан...
Осталось итераций: 51
# Итерация 3. Р’РѕРґР°_Рё_РЅР°РїРёС‚РєРё записан...
Осталось итераций: 50
